In [ ]:
#data files:
#data_path = "/Users/amy/Desktop/coax/group/n_324/"
data_path = "../data/"
anat_vol_file = "PIP_n324_Freesurfer_aseg.csv"
anat_SA_file = "PIP_n324_Freesurfer_aparc2009_SA.csv"
anat_thickness_file = "PIP_n324_Freesurfer_aparc2009_thickness.csv"
rs_FC_file = "PIP_n324_rs-FC_FD_residuals.csv"
#covariates_file = "PIP_n331_Framingham.csv"
covariates_file = "PIP_n324_Framingham.csv"
data_file_names = [rs_FC_file, anat_SA_file, anat_thickness_file, anat_vol_file, covariates_file]
#demographics_file = "/Users/amy/Desktop/coax/group/n_324/PIP_n324_demographics_modified.csv"
demographics_file = "PIP_n324_demographics_modified.csv"
#subj_list = "/Users/amy/Desktop/coax/group/n_324/subj_list_n324.csv"
subj_list = "subj_list_n324.csv"

In [ ]:
subject_list = pd.read_csv(subj_list, header=None)
str_subject_list = subject_list.astype(str)
n_subj = subject_list[0].count()

In [ ]:
def extract_outcome_measure(n_subj, subject_list, outcome_file, ID_column_name, outcome_column_name):

    subjects = subject_list.values.reshape(n_subj,)
    tmp_df = pd.read_csv(outcome_file, usecols=[ID_column_name, outcome_column_name])
    y_tmp = tmp_df.loc[tmp_df[ID_column_name].isin(subjects)]
    y = y_tmp[outcome_column_name].values
    print(outcome_column_name + " loaded")

    return y, y_tmp #df coming back with labels

In [ ]:
#extract response variable - IMT
outcome_column_name = "mavgimt"
ID_column_name = "ID"
y, y_df = extract_outcome_measure(n_subj, str_subject_list, demographics_file, ID_column_name, outcome_column_name)

In [ ]:
xticks = np.arange(0.4,0.95,0.05)

fig, ax = plt.subplots(figsize=(10,7))

ax = pt.RainCloud(data=y, orient="h", palette='Blues') #BuGn
#ax.set_xlabel("IMT", fontsize='x-large')
ax.set_xlabel(r'IMT (mm)', size=30, weight = 'bold')
ax.yaxis.set_ticklabels([])
plt.xticks(np.round(xticks,2))
#ax.xaxis.set_ticklabels(np.round(xticks,2))
ax.tick_params(axis='x', labelsize=15)
plt.tight_layout()
plt.savefig('/Users/amy/Desktop/coax/IMT_raincloud_111621.png', dpi=600)

In [ ]:
#extract framingham indices - HCHD, HeartAge, TotalPoints
ID_column_name = "ePrime.id"

TotalPoints_column_name = "TotalPoints"
TotalPoints_tempX, TotalPoints_df_temp = extract_outcome_measure(n_subj, str_subject_list, data_path+covariates_file, 
                                                      ID_column_name, TotalPoints_column_name)

HCHD_column_name = "HCHD"
HCHD_tempX, HCHD_df_temp = extract_outcome_measure(n_subj, str_subject_list, data_path+covariates_file, 
                                        ID_column_name, HCHD_column_name)

HeartAge_column_name = "HeartAge"
HeartAge_tempX, HeartAge_df_temp = extract_outcome_measure(n_subj, str_subject_list, data_path+covariates_file, 
                                                ID_column_name, HeartAge_column_name)

In [ ]:
#sort df framingham data by ID
TotalPoints_df = TotalPoints_df_temp.sort_values(by=[ID_column_name])
HCHD_df = HCHD_df_temp.sort_values(by=[ID_column_name])
HeartAge_df = HeartAge_df_temp.sort_values(by=[ID_column_name])

#get sorted numpy data from df
TotalPoints_temp = TotalPoints_df[TotalPoints_column_name].values
HCHD_temp = HCHD_df[HCHD_column_name].values
HeartAge_temp = HeartAge_df[HeartAge_column_name].values

In [ ]:
# needs to be fixed (got rid of excess subjs in file)
i_nan = [57,61,251,273,283,284]
np.put(TotalPoints_temp, i_nan, [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
TotalPoints = TotalPoints_temp.astype(np.float)

In [ ]:
#create covariate channel
cov_data = TotalPoints.reshape(-1, 1)
cov_data.shape

In [ ]:
#import matplotlib
#matplotlib.rc_file_defaults()

fig, ax = plt.subplots(figsize=(10,7))
ax = pt.RainCloud(data=cov_data, orient="h", palette='Greens')
#ax.set_xlabel("FRS", fontsize='x-large')
ax.set_xlabel(r'FRS', size=30, weight = 'bold')
ax.tick_params(axis='x', labelsize=15)
ax.yaxis.set_ticklabels([])
plt.tight_layout()
plt.savefig('/Users/amy/Desktop/coax/FRS_raincloud_092121.png', dpi=600)



### Brain age

In [ ]:
# need to save results with pickl and reload here

In [ ]:
#PLOT RESULTS - seaborn aesthetics

sns.set()

fig, ax = plt.subplots(figsize=(10,10))


age_observed = np.concatenate((age_train, age_test))
age_predicted_overall = np.concatenate((age_predicted_cv2, age_predicted_test2))

#calculate line of best fit
slope, line_intercept, r_value, p_value, std_err = linregress(age_observed, age_predicted_overall)
line = slope*age_observed + line_intercept


ax.scatter(age_observed, age_predicted_overall, color='darkorange', s=16)
ax.plot(age_observed, line, color='royalblue')
#x0,x1 = ax.get_xlim()
#y0,y1 = ax.get_ylim()

ax.set_xlabel(r'Chronological age', size=17, weight = 'bold')
ax.set_ylabel(r'Predicted brain age', size=17, weight = 'bold')
#fig.suptitle('Correlation between observed and predicted intima media thickness (IMT)')
ax.set_xlim((29,52))
ax.set_ylim((29,52))
#ax.set_xticks(np.arange(0.4,0.9,0.08))
#ax.set_yticks(np.arange(0.4,0.9,0.08))
ax.set_aspect('equal')

ax.tick_params(axis='x', labelsize = 15)
ax.tick_params(axis='y', labelsize = 15)
plt.tight_layout()
plt.savefig("/Users/amy/Desktop/coax/brain_age_110321.png", dpi=600)

In [ ]:
print("r:", r_value)
print("p:", p_value)

## FRS vs. IMT

In [ ]:
# load pickled data

In [ ]:
FRS = cov_data_imputed.reshape(-1,)

In [ ]:
x_ticks = np.arange(-15,22.5,2.5)
y_ticks = np.arange(0.4,0.95,0.05)
x_ticks

In [ ]:
sns.set()

fig, axes = plt.subplots(figsize=(10,10))

#calculate line of best fit
slope, line_intercept, r_value, p_value, std_err = linregress(FRS, y)
line = slope*FRS + line_intercept

axes.scatter(FRS, y, color='darkorange', s=16)
axes.plot(FRS, line, color='royalblue')

axes.set_xlabel(r'FRS', size=30, weight = 'bold')
axes.set_ylabel(r'IMT (mm)', size=30, weight = 'bold')
axes.tick_params(axis='x', labelsize = 15)
axes.tick_params(axis='y', labelsize = 15)
plt.yticks(np.round(y_ticks,2))
#plt.xticks(np.round(x_ticks,2))
plt.tight_layout()
plt.savefig("/Users/amy/Desktop/coax/FRS_IMT_111621.png", dpi=600)



#ax.xaxis.set_ticklabels(np.round(xticks,2))

In [ ]:
r_value
p_value

In [ ]:
clr_palette = ['blue','blue','blue','blue','blue',
               'orange','orange','orange','orange','orange','orange','orange','orange','orange','orange','orange',
               'green','green','green','green','green','green','green','green','green','green','green','green',
               'green','green','green']
#clr_palette

In [ ]:
import matplotlib
matplotlib.rc_file_defaults()

In [ ]:
f = open('/Users/amy/Desktop/coax/group/n_324/mc_comb_rmse.pckl', 'rb')
[rmse, str_subset] = pickle.load(f) # rmse (+ str_subset for channel combination order)
f.close()

In [ ]:
f = open('/Users/amy/Desktop/coax/group/n_324/mc_single_channel_predictions_rmse.pckl', 'rb')
[single_channel_rmse] = pickle.load(f) 
f.close()

In [ ]:
#v2 with (1,2,3,4,5) instead of (0,1,2,3,4)

#change to aascending order separated by FI
str_brain = []
str_FI = []
brain_i = []
FI_i = []
for i,sub in enumerate(str_subset_v2):
    if '5' in sub:
        str_FI.append(sub)
        FI_i.append(i)
    else:
        str_brain.append(sub)
        brain_i.append(i)

rmse_brain = rmse[:,brain_i]
rmse_FI = rmse[:,FI_i]

comb_r2_FI = comb_r2[:,FI_i]
comb_r2_brain = comb_r2[:,brain_i]

comb_bic_FI = comb_bic[:,FI_i]
comb_bic_brain = comb_bic[:,brain_i]

r_brain = comb_r[:,brain_i]
r_FI = comb_r[:,FI_i]



In [ ]:
#v2 with (1,2,3,4,5) instead of (0,1,2,3,4)

#rmse sorted in order of subset (i.e. increasing complexity) with FI separated

single_channels = ['resting-state FC', 'cortical SA', 'cortical thickness', 'subcortical volume', 'FRS']

mc_cc_rmse_dict = dict()

for  j, name in enumerate(single_channels):
    mc_cc_rmse_dict[name] = single_channel_rmse[:, j] 

for  i, name in enumerate(str_brain):
    mc_cc_rmse_dict[name] = rmse_brain[:, i] 
    
for  k, name in enumerate(str_FI):
    mc_cc_rmse_dict[name] = rmse_FI[:, k] 


mc_cc_rmse_df = pd.melt(pd.DataFrame(mc_cc_rmse_dict))

In [ ]:
fig, ax = plt.subplots(figsize=(25,15))

sns.stripplot(y='value', 
              x='variable', 
              data=mc_cc_rmse_df, 
              size= 6,
              alpha=0.25,
              jitter=0.2,
              linewidth = 1,
              color = 'white',
              edgecolor ='black',
              ax = ax)

sns.pointplot(y='value', 
              x='variable', 
              data=mc_cc_rmse_df, 
              estimator = np.median, 
              seed = 0, 
              color='black',
              #errwidth=6,
              capsize=.05,
              linestyles='--',
              #scale=0.8,
              #errcolor=".05",
              ax=ax)

sns.barplot(y='value', 
            x='variable', 
            data=mc_cc_rmse_df, 
            estimator = np.median,
            #capsize=0,
            #linewidth=0.,#2.5,
            #errwidth = 0,
            #errcolor=".05", edgecolor=".05",
            alpha=0.75,
            palette = clr_palette,
            ax = ax)

ax.axhline(0.0848, ls='--', lw=4)
ax.set_ylabel(r'Root mean squared error (RMSE)', size=30, weight = 'bold')
ax.set_xlabel(r'Channel (single or combination)', size=30, weight = 'bold')

ax.tick_params(axis='y', labelsize=30)
ax.set_xticklabels(mc_cc_rmse_dict.keys(), rotation='vertical', size=30)
#ax.set_title("", size=50, weight = 'bold', loc='center')
plt.ylim([0.07, 0.105])
plt.tight_layout()
plt.savefig("/Users/amy/Desktop/coax/monte_carlo_channel_combination_rmse_092521.png", dpi=600)

In [ ]:
single_channel_rmse_median = np.median(single_channel_rmse,axis=0)
print(single_channel_rmse_median)

In [ ]:
rmse_brain_median = np.median(rmse_brain,axis=0)
print(rmse_brain_median)
print("min", np.min(rmse_brain_median),"max", np.max(rmse_brain_median))

In [ ]:
rmse_FI_median = np.median(rmse_FI,axis=0)
print(rmse_FI_median)
print("min", np.min(rmse_FI_median),"max", np.max(rmse_FI_median))

In [ ]:
rmse_brain_mean = np.mean(rmse_brain,axis=0)
print(rmse_brain_mean)
print("min", np.min(rmse_brain_mean),"max", np.max(rmse_brain_mean))

In [ ]:
rmse_FI_mean = np.mean(rmse_FI,axis=0)
print(rmse_FI_mean)
print("min", np.min(rmse_FI_mean),"max", np.max(rmse_FI_mean))

In [ ]:
f = open('/Users/amy/Desktop/coax/group/n_324/mc_comb_metrics.pckl', 'rb')
[comb_r, comb_r2, comb_bf, str_subset] = pickle.load(f) # r, r2, bf (+ str_subset for channel combination order)
f.close()

In [ ]:
# single channel r, r2
f = open('/Users/amy/Desktop/coax/group/n_324/mc_single_channel_r_r2.pckl', 'rb')
[single_channel_r, single_channel_r2] = pickle.load(f)
f.close()

In [ ]:
r_brain = comb_r[:,brain_i]
r_FI = comb_r[:,FI_i]

r2_brain = comb_r2[:,brain_i]
r2_FI = comb_r2[:,FI_i]

In [ ]:
#r sorted in order of subset (i.e. increasing complexity) with FI separated

single_channels = ['resting-state FC', 'cortical SA', 'cortical thickness', 'subcortical volume', 'FRS']

mc_cc_r_dict = dict()

for  j, name in enumerate(single_channels):
    mc_cc_r_dict[name] = single_channel_r[:, j] 

for  i, name in enumerate(str_brain):
    mc_cc_r_dict[name] = r_brain[:, i] 
    
for  k, name in enumerate(str_FI):
    mc_cc_r_dict[name] = r_FI[:, k] 


mc_cc_r_df = pd.melt(pd.DataFrame(mc_cc_r_dict))

In [ ]:
fig, ax = plt.subplots(figsize=(25,15))

sns.stripplot(y='value', 
              x='variable', 
              data=mc_cc_r_df, 
              size= 6,
              alpha=0.25,
              jitter=0.2,
              linewidth = 1,
              color = 'white',
              edgecolor ='black',
              ax = ax)

sns.pointplot(y='value', 
              x='variable', 
              data=mc_cc_r_df, 
              estimator = np.median, 
              seed = 0, 
              color='black',
              #errwidth=6,
              capsize=.05,
              linestyles='--',
              #scale=0.8,
              #errcolor=".05",
              ax=ax)

sns.barplot(y='value', 
            x='variable', 
            data=mc_cc_r_df, 
            estimator = np.median,
            #capsize=0,
            #linewidth=0.,#2.5,
            #errwidth = 0,
            #errcolor=".05", edgecolor=".05",
            alpha=0.75,
            palette = clr_palette,
            ax = ax)

ax.set_ylabel(r'Pearson correlation coefficient, r', size=30, weight = 'bold')
ax.set_xlabel(r'Channel (single or combination)', size=30, weight = 'bold')

ax.tick_params(axis='y', labelsize=30)
ax.set_xticklabels(mc_cc_r_dict.keys(), rotation='vertical', size=30)
#ax.set_title("", size=50, weight = 'bold', loc='center')
#plt.ylim([0.07, 0.105])
plt.tight_layout()
plt.savefig("/Users/amy/Desktop/coax/monte_carlo_channel_combination_r_092521.png", dpi=600)

In [ ]:
single_channel_r_median = np.median(single_channel_r,axis=0)
print(single_channel_r_median)

In [ ]:
r_brain_median = np.median(r_brain,axis=0)
print(r_brain_median)
print("min", np.min(r_brain_median),"max", np.max(r_brain_median))

In [ ]:
r_FI_median = np.median(r_FI,axis=0)
print(r_FI_median)
print("min", np.min(r_FI_median),"max", np.max(r_FI_median))

In [ ]:
r_brain_mean = np.mean(r_brain,axis=0)
print(r_brain_mean)
print("min", np.min(r_brain_mean),"max", np.max(r_brain_mean))

In [ ]:
r_FI_mean = np.mean(r_FI,axis=0)
print(r_FI_mean)
print("min", np.min(r_FI_mean),"max", np.max(r_FI_mean))

In [ ]:
#r2 sorted in order of subset (i.e. increasing complexity) with FI separated

single_channels = ['resting-state FC', 'cortical SA', 'cortical thickness', 'subcortical volume', 'FRS']

mc_cc_r2_dict = dict()

for  j, name in enumerate(single_channels):
    mc_cc_r2_dict[name] = single_channel_r2[:, j] 

for  i, name in enumerate(str_brain):
    mc_cc_r2_dict[name] = comb_r2_brain[:, i] 
    
for  k, name in enumerate(str_FI):
    mc_cc_r2_dict[name] = comb_r2_FI[:, k] 


mc_cc_r2_df = pd.melt(pd.DataFrame(mc_cc_r2_dict))

In [ ]:
fig, ax = plt.subplots(figsize=(25,15))

sns.stripplot(y='value', 
              x='variable', 
              data=mc_cc_r2_df, 
              size= 6,
              alpha=0.25,
              jitter=0.2,
              linewidth = 1,
              color = 'white',
              edgecolor ='black',
              ax = ax)

sns.pointplot(y='value', 
              x='variable', 
              data=mc_cc_r2_df, 
              estimator = np.median, 
              seed = 0, 
              color='black',
              #errwidth=6,
              capsize=.05,
              linestyles='--',
              #scale=0.8,
              #errcolor=".05",
              ax=ax)

sns.barplot(y='value', 
            x='variable', 
            data=mc_cc_r2_df, 
            estimator = np.median,
            #capsize=0,
            #linewidth=0.,#2.5,
            #errwidth = 0,
            #errcolor=".05", edgecolor=".05",
            alpha=0.75,
            palette = clr_palette,
            ax = ax)

ax.set_ylabel(r'Coefficient of determination, $R^2$', size=30, weight = 'bold')
ax.set_xlabel(r'Channel (single or combination)', size=30, weight = 'bold')

ax.tick_params(axis='y', labelsize=30)
ax.set_xticklabels(mc_cc_r2_dict.keys(), rotation='vertical', size=30)
#ax.set_title("", size=50, weight = 'bold', loc='center')
#plt.ylim([0.07, 0.105])
plt.tight_layout()
plt.savefig("/Users/amy/Desktop/coax/monte_carlo_channel_combination_r2_102321.png", dpi=600)

In [ ]:
mc_r2_rsFC_mean = np.mean(mc_cc_r2_dict['resting-state FC'])
mc_r2_rsFC_mean

In [ ]:
mc_r2_rsFC_median = np.median(mc_cc_r2_dict['resting-state FC'])
mc_r2_rsFC_median

In [ ]:
mc_r2_SA_mean = np.mean(mc_cc_r2_dict['cortical SA'])
mc_r2_SA_mean

In [ ]:
mc_r2_SA_median = np.median(mc_cc_r2_dict['cortical SA'])
mc_r2_SA_median

In [ ]:
mc_r2_ct_mean = np.mean(mc_cc_r2_dict['cortical thickness'])
mc_r2_ct_mean

In [ ]:
mc_r2_ct_median = np.median(mc_cc_r2_dict['cortical thickness'])
mc_r2_ct_median

In [ ]:
mc_r2_vol_mean = np.mean(mc_cc_r2_dict['subcortical volume'])
mc_r2_vol_mean

In [ ]:
mc_r2_vol_median = np.median(mc_cc_r2_dict['subcortical volume'])
mc_r2_vol_median

In [ ]:
mc_r2_frs_mean = np.mean(mc_cc_r2_dict['FRS'])
mc_r2_frs_mean

In [ ]:
mc_r2_frs_median = np.median(mc_cc_r2_dict['FRS'])
mc_r2_frs_median

In [ ]:
np.mean(comb_r2,axis=0)

In [ ]:
np.median(comb_r2,axis=0)

In [ ]:
# comb_bic + str_subset
f = open('/Users/amy/Desktop/coax/group/n_324/mc_comb_bic.pckl', 'rb')
[comb_bic, str_subset] = pickle.load(f)
f.close()

# step1_bic 
f = open('/Users/amy/Desktop/coax/group/n_324/mc_single_channel_bic.pckl', 'rb')
[step1_bics] = pickle.load(f)
f.close()

In [ ]:
#bic sorted in order of subset (i.e. increasing complexity) with FI separated

single_channels = ['resting-state FC', 'cortical SA', 'cortical thickness', 'subcortical volume', 'FRS']

mc_cc_bic_dict = dict()

for  j, name in enumerate(single_channels):
    mc_cc_bic_dict[name] = step1_bics[:, j] 

for  i, name in enumerate(str_brain):
    mc_cc_bic_dict[name] = comb_bic_brain[:, i] 
    
for  k, name in enumerate(str_FI):
    mc_cc_bic_dict[name] = comb_bic_FI[:, k] 


mc_cc_bic_df = pd.melt(pd.DataFrame(mc_cc_bic_dict))

In [ ]:
np.median(step1_bics, axis=0)

In [ ]:
np.median(comb_bic, axis=0)

In [ ]:
import matplotlib.gridspec as gridspec


ylim = [80000, 84000]
ylim2  = [-400, 400]
ylimratio = (ylim[1]-ylim[0])/(ylim2[1]-ylim2[0]+ylim[1]-ylim[0])
ylim2ratio = (ylim2[1]-ylim2[0])/(ylim2[1]-ylim2[0]+ylim[1]-ylim[0])

gs = gridspec.GridSpec(2, 1, height_ratios=[ylim2ratio, ylimratio])
fig = plt.figure(figsize=(25,15))
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])

sns.stripplot(y='value', 
              x='variable', 
              data=mc_cc_bic_df, 
              size= 6,
              alpha=0.25,
              jitter=0.2,
              linewidth = 1,
              color = 'white',
              edgecolor ='black',
              ax=ax1)

sns.pointplot(y='value', 
              x='variable', 
              data=mc_cc_bic_df, 
              estimator = np.median, 
              seed = 0, 
              color='black',
              #errwidth=6,
              capsize=.05,
              linestyles='--',
              #scale=0.8,
              #errcolor=".05",
              ax=ax1)

sns.barplot(y='value', 
            x='variable', 
            data=mc_cc_bic_df, 
            estimator = np.median,
            #capsize=0,
            #linewidth=0.,#2.5,
            #errwidth = 0,
            #errcolor=".05", edgecolor=".05",
            alpha=0.75,
            palette = clr_palette,
            ax=ax1)

sns.stripplot(y='value', 
              x='variable', 
              data=mc_cc_bic_df, 
              size= 6,
              alpha=0.25,
              jitter=0.2,
              linewidth = 1,
              color = 'white',
              edgecolor ='black',
              ax=ax2)

sns.pointplot(y='value', 
              x='variable', 
              data=mc_cc_bic_df, 
              estimator = np.median, 
              seed = 0, 
              color='black',
              #errwidth=6,
              capsize=.05,
              linestyles='--',
              #scale=0.8,
              #errcolor=".05",
              ax=ax2)

sns.barplot(y='value', 
            x='variable', 
            data=mc_cc_bic_df, 
            estimator = np.median,
            #capsize=0,
            #linewidth=0.,#2.5,
            #errwidth = 0,
            #errcolor=".05", edgecolor=".05",
            alpha=0.75,
            palette = clr_palette,
            ax=ax2)

ax1.set_ylim(ylim) #top subplot
ax2.set_ylim(ylim2) #bottom subplot

plt.subplots_adjust(hspace=0.01)

ax1.spines['bottom'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax1.xaxis.tick_top()
ax1.xaxis.set_ticks_position('none')
ax1.tick_params(labeltop=False)
ax2.xaxis.tick_bottom()

ax1.tick_params(axis='y', labelsize=25)
ax2.tick_params(axis='y', labelsize=25)
ax2.set_xticklabels(mc_cc_bic_dict.keys(), rotation='vertical', size=30)

ax1.set_xlabel('')
ax1.set_ylabel('')
ax2.set_xlabel(r'Channel (single or combination)', size=30, weight = 'bold')
ax2.set_ylabel(r'Bayesian information criterion', size=30, weight = 'bold')
ax2.yaxis.set_label_coords(0.05, 0.65, transform=fig.transFigure)

kwargs = dict(color='k', clip_on=False)
xlim = ax1.get_xlim()
dx = .02*(xlim[1]-xlim[0])
dy = .01*(ylim[1]-ylim[0])/ylim2ratio
ax1.plot((xlim[0]-dx,xlim[0]+dx), (ylim[0]-dy,ylim[0]+dy), **kwargs)
ax1.plot((xlim[1]-dx,xlim[1]+dx), (ylim[0]-dy,ylim[0]+dy), **kwargs)
dy = .01*(ylim2[1]-ylim2[0])/ylimratio
ax2.plot((xlim[0]-dx,xlim[0]+dx), (ylim2[1]-dy,ylim2[1]+dy), **kwargs)
ax2.plot((xlim[1]-dx,xlim[1]+dx), (ylim2[1]-dy,ylim2[1]+dy), **kwargs)
ax1.set_xlim(xlim)
ax2.set_xlim(xlim)







#ax.set_title("", size=50, weight = 'bold', loc='center')

plt.tight_layout()
#plt.savefig("/Users/amy/Desktop/coax/monte_carlo_channel_combination_bic_102521.png", dpi=600)

## OLD

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import ptitprince as pt
import seaborn as sns

from nilearn import datasets, image, plotting
from scipy.stats import linregress, pearsonr, spearmanr

### Load Data

In [7]:
def load_combined_data(data_path, filename):

    if 'rs-FC' in filename:
        X_df = pd.read_csv(data_path + filename, header=None)
    else:
        X_df_temp = pd.read_csv(data_path + filename) #header=None
        X_df = X_df_temp.drop(X_df_temp.columns[0], axis=1)
        
    X = X_df.values
    print(filename + " loaded")

    return X, X_df